# **Tiled Matrix Multiplication Project Assignment by Wael KABOUK, ID:Y235050005**

##Preparing CUDA Environment

In [24]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-wtgvjn63
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-wtgvjn63
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


## Implementing The Code And Writing it To A File



In [35]:
%%writefile tiledMatrixMult.cu

#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include <cuda_runtime.h>
#include <cstdlib>
#include <chrono>
#include <time.h>
#define TILE_SIZE 32
#define CHECK(call)                                            \
{                                                               \
    const cudaError_t error = call;                             \
    if (error != cudaSuccess)                                   \
    {                                                           \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__); \
        fprintf(stderr, "code: %d, reason: %s\n", error,        \
        cudaGetErrorString(error));                     \
        exit(EXIT_FAILURE);                                             \
    }                                                           \
}

// Matrix multiplication kernel with shared memory
__global__ void matrixMultiplyGPU(const int* A, const int* B, int* C, int N)
{
    // Declare shared memory
    __shared__ int shared_A[TILE_SIZE][TILE_SIZE];
    __shared__ int shared_B[TILE_SIZE][TILE_SIZE];

    // Calculate thread indices
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Initialize the result in C to 0
    int result = 0;

    // Loop over tiles
    for (int t = 0; t < N / TILE_SIZE; ++t)
    {
        // Load tiles into shared memory
        shared_A[threadIdx.y][threadIdx.x] = A[row * N + t * TILE_SIZE + threadIdx.x];
        shared_B[threadIdx.y][threadIdx.x] = B[(t * TILE_SIZE + threadIdx.y) * N + col];

        // Synchronize threads to ensure tiles are loaded
        __syncthreads();

        // Multiply the tiles and accumulate the result
        for (int k = 0; k < TILE_SIZE; ++k)
        {
            result += shared_A[threadIdx.y][k] * shared_B[k][threadIdx.x];
        }

        // Synchronize threads before loading the next tiles
        __syncthreads();
    }

    // Write the result to the output matrix
    C[row * N + col] = result;
}

// Function to initialize a matrix with random values
void initializeMatrix(int* matrix, int size)
{
    for (int i = 0; i < size; ++i)
    {
        matrix[i] = rand() % 100;
    }
}

// Function to print the elements of a matrix
void printMatrix(int* matrix, int rows, int cols)
{
    for (int i = 0; i < rows; ++i)
    {
        for (int j = 0; j < cols; ++j)
        {
            printf("%d ", matrix[i * cols + j]);
        }
        printf("\n");
    }
    printf("\n");
}

// Function to perform matrix multiplication on CPU
void matrixMultiplyCPU(int *h_A, int *h_B, int *h_C_cpu, int N){

  for (int i = 0; i < N; ++i)
    {
        for (int j = 0; j < N; ++j)
        {
            int sum = 0;
            for (int k = 0; k < N; ++k)
            {
                sum += h_A[i * N + k] * h_B[k * N + j];
            }
            h_C_cpu[i * N + j] = sum;
        }
    }

}

int main(int argc, char** argv)
{
    // Initialize random seed
    srand((unsigned)time(NULL));

    // Matrix size and size calculation
    int N = atoi(argv[1]);
    int numElements = N * N;
    size_t size = numElements * sizeof(int);

    printf("Matrix multiplication of %d x %d matrices\n", N, N);

    // Allocate host input and output matrices
    int* h_A = (int*)malloc(size);
    int* h_B = (int*)malloc(size);
    int* h_C_cpu = (int*)malloc(size);
    int* h_C_gpu = (int*)malloc(size);

    // Verify host matrix allocations
    if (h_A == NULL || h_B == NULL || h_C_cpu == NULL || h_C_gpu == NULL)
    {
        fprintf(stderr, "Failed to allocate host matrices!\n");
        exit(EXIT_FAILURE);
    }

    // Initialize host input matrices
    initializeMatrix(h_A, numElements);
    initializeMatrix(h_B, numElements);

    // Allocate device input and output matrices
    int* d_A = NULL;
    int* d_B = NULL;
    int* d_C = NULL;

    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copy host input matrices to device
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Define block and grid sizes
    dim3 blockSize(TILE_SIZE, TILE_SIZE);
    dim3 gridSize((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);

    // CUDA event setup for timing
    cudaEvent_t start, stop;
    float elapsedTime;

    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Record CUDA event start time
    cudaEventRecord(start, 0);

    // Launch the Matrix Multiply CUDA Kernel
    matrixMultiplyGPU << <gridSize, blockSize >> > (d_A, d_B, d_C, N);
    cudaDeviceSynchronize();

    // Record CUDA event stop time
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    // Calculate and print GPU elapsed time
    cudaEventElapsedTime(&elapsedTime, start, stop);
    printf("Time on device: %f ms\n", elapsedTime);

    // Copy device result matrix to host
    cudaMemcpy(h_C_gpu, d_C, size, cudaMemcpyDeviceToHost);

    // Record the start time for CPU
    clock_t cpuStart, cpuStop;
    float cpuDuration;

    // Record the start time for CPU
    cpuStart = clock();

    // Perform matrix multiplication on CPU
    matrixMultiplyCPU(h_A, h_B, h_C_cpu, N);

    // Record the stop time for CPU
    cpuStop = clock();

    // Calculate elapsed time on the CPU
    cpuDuration = ((float)(cpuStop - cpuStart)) * 1000.0 / CLOCKS_PER_SEC;
    printf("Time on CPU: %f ms\n", cpuDuration);

    // Calculate and print speedup
    float speedup = cpuDuration / elapsedTime;
    printf("Speed-up: %.2f\n", speedup);

    // Verify the result matrices
    for (int i = 0; i < numElements; ++i)
    {
        if (abs(h_C_cpu[i] - h_C_gpu[i]) > 1e-5)
        {
            fprintf(stderr, "Result verification failed at element %d!\n", i);
            exit(EXIT_FAILURE);
        }
    }

    printf("Test PASSED\n");

    // Free device global memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C_cpu);
    free(h_C_gpu);

    printf("Done\n");
    return 0;
}


Overwriting tiledMatrixMult.cu


## Compiling The code

In [36]:
!nvcc -o tiledMatrixMult tiledMatrixMult.cu

## Executing the code with an input value of N = 256

In [37]:
!./tiledMatrixMult 256

Matrix multiplication of 256 x 256 matrices
Time on device: 0.291808 ms
Time on CPU: 57.529999 ms
Speed-up: 197.15
Test PASSED
Done


## Executing the code with an input value of N = 512


In [38]:
!./tiledMatrixMult 512

Matrix multiplication of 512 x 512 matrices
Time on device: 0.837216 ms
Time on CPU: 753.812988 ms
Speed-up: 900.38
Test PASSED
Done


## Executing the code with an input value of N = 1024

In [39]:
!./tiledMatrixMult 1024

Matrix multiplication of 1024 x 1024 matrices
Time on device: 5.410144 ms
Time on CPU: 8158.627930 ms
Speed-up: 1508.02
Test PASSED
Done


## Conclusion
In summary, the CUDA-accelerated matrix multiplication implementation presented in this code leverages the parallel processing capabilities of GPUs and employs shared memory to enhance computational efficiency. The comparison of execution times between the GPU and CPU versions highlights the substantial speedup achieved by utilizing GPU resources, particularly evident when dealing with larger matrix sizes.

The reliability of the results, as evidenced by the agreement between the CPU and GPU outcomes, affirms the correctness of the parallelized matrix multiplication algorithm. The implementation's ability to handle matrices of varying sizes underscores its adaptability to diverse computational demands.

The successful execution of matrix multiplication on both GPU and CPU platforms underscores the versatility and effectiveness of the parallelized approach. This code provides a foundation for harnessing GPU acceleration in numerical computations, paving the way for enhanced performance in a wide range of applications involving matrix operations